# Multiple answer cleanup
Attempt at dealing with fields that contain comma-separated lists of answers from questions with multiple-select answer options that also allow respondents to enter their own 'other' answer.

This notebook assumes the file named `responses_scrubbed_tagged.csv` has already been created by running the notebook named `free_text_tagging.ipynb`.  This notebook then creates a new data file called `responses_complete.csv` which has the complete set of final data.

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, Markdown, Latex

## Load tagged data set
This notebook assumes you have already scrubbed and tagged the data using the `free_text_tagging.ipynb` notebook first.

In [2]:
df = pd.read_csv('responses_scrubbed_tagged.csv', index_col=None, parse_dates=['date'])
df.sample(n=10) #random sample of rows

,date,neighborhood,demographic,child_bus_freq,child_bike_freq,child_driven_freq,child_drive_freq,child_walk_freq,walk_freq,bike_freq,...,no_bike_reason_2,no_bike_reason_3,drive_reason_0,drive_reason_1,drive_reason_2,drive_reason_3,drive_reason_4,drive_reason_5,problem_tags,suggestion_tags
388,2019-07-08 10:29:53,Upper Village,Adult with school kid,Once in a while,Once in a while,Once in a while,Rarely or never - for other reasons,Once in a while,Once every few days,Once in a while,...,NaN,NaN,Safety concerns with walking,NaN,NaN,NaN,NaN,NaN,"dummy light,crosswalk condition,sidewalk condi...","pedestrian-only upper village,dedicated pedest..."
397,2019-07-08 16:58:42,Harmon,Adult with school kid,Rarely or never,Rarely or never - for other reasons,"Every day, or almost every day",Rarely or never - my children are too young,Rarely or never - my children are too young,Once in a while,Rarely or never,...,NaN,NaN,Safety concerns with walking,Safety concerns with bicycling,NaN,NaN,NaN,NaN,"missing crosswalks,speeding,driver awareness,c...","better lighting,educate cyclists,more crosswal..."
424,2019-07-09 13:51:38,Harmon,Adult with school kid,Once every few days,Rarely or never - for other reasons,"Every day, or almost every day",Rarely or never - my children are too young,Rarely or never - my children are too young,Once in a while,Once in a while,...,NaN,NaN,I don't have time to walk or bicycle,NaN,NaN,NaN,NaN,NaN,gottwald circle,more stop signs
495,2019-07-14 21:51:42,Quaker Ridge,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Rarely or never,Rarely or never,...,NaN,NaN,I don't have time to walk or bicycle,NaN,NaN,NaN,NaN,NaN,"cpa,road surface",NaN
44,2019-07-03 15:48:29,Harmon,Adult with school kid,"Every day, or almost every day",Rarely or never - my children are too young,Rarely or never,Rarely or never - my children are too young,Once in a while,"Every day, or almost every day",Rarely or never,...,NaN,NaN,Safety concerns with walking,NaN,NaN,NaN,NaN,NaN,"gottwald circle,missing sidewalks,cpa,shoprite...","maintain crosswalks, more crosswalks, maintain..."
91,2019-07-03 16:19:49,Harmon,Adult with school kid,Rarely or never,Rarely or never - for other reasons,"Every day, or almost every day",Rarely or never - for other reasons,"Every day, or almost every day","Every day, or almost every day",Once in a while,...,NaN,NaN,Safety concerns with bicycling,NaN,NaN,NaN,NaN,NaN,"missing sidewalks, cet, cleveland, five corner...","more sidewalks,more crosswalks,better lighting..."
229,2019-07-03 23:08:46,Sunset Park,College kid,NaN,NaN,NaN,NaN,NaN,Rarely or never,"Every day, or almost every day",...,NaN,NaN,I don't have time to walk or bicycle,NaN,NaN,NaN,NaN,NaN,"aggressive driving,driver awareness,municipal ...","beautify underpass on cpa,bike lanes"
302,2019-07-04 16:45:45,Old Post Road North,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Once every few days,Once every few days,...,NaN,NaN,My own personal preference,NaN,NaN,NaN,NaN,NaN,"pedestrians in roads,cyclists breaking rules","educate cyclists,educate pedestrians"
140,2019-07-03 17:26:59,Albany Post Road,Senior,NaN,NaN,NaN,NaN,NaN,Rarely or never,Rarely or never,...,NaN,NaN,My own personal preference,NaN,NaN,NaN,NaN,NaN,five corners,"more traffic lights,traffic light at five corners"
511,2019-07-18 09:46:57,Upper Village,Adult with school kid,"Every day, or almost every day",Rarely or never - my children are too young,Rarely or never,Rarely or never - my children are too young,Rarely or never - my children are too young,Once in a while,Rarely or never,...,NaN,NaN,I don't have time to walk or bicycle,NaN,NaN,NaN,NaN,NaN,"cpa,s riverside,aggressive driving",bike lanes


## Answer options to each question
A dictionary containing lists of all the questions and the answer options that the user was asked to select from for each question in the Google Form.

In [3]:
# a dictionary of columns that contain comma-separated multiple answer options
multiple_answer_options = {
    'schools': ['CET', 'PVC', 'CHHS', 'Homeschooled'],
    'child_drive_reason': ['No - we do not drive, or prefer not to drive', 'Our own personal preference', 'Lack of available busing where we live', 'The bus schedule does not match our schedule', 'Safety concerns with buses', 'Safety concerns with walking', 'Safety concerns with bicycling', "My child's health condition"],
    'child_no_walk_reason': ['No - they walk a lot', 'My child does not like to walk', 'We live too far to walk', "We don't have time to walk", 'Fear of dangerous driving', 'Lack of adequate sidewalks', 'Lack of adequate crosswalks at busy intersections', 'Lack of crossing guards at busy intersections', "My child's health condition", 'Visually unappealing route'],
    'child_no_bike_reason': ['No - they bicycle a lot', 'My child does not like to bicycle', 'My child is too young to bicycle', 'We live too far to bicycle', 'Our own personal preference', 'Fear of dangerous driving', 'Lack of adequate bike lanes', "My child's health condition", 'Visually unappealing route', 'Hills'],
    'no_walk_reason': ['No - I walk a lot', 'I live too far to walk', 'I do not like to walk', "I don't have time to walk", 'Fear of dangerous driving', 'Lack of adequate sidewalks', 'Lack of adequate crosswalks at intersections', 'Lack of crossing guards at intersections', "My health condition", 'My own personal preference', 'Visually unappealing'], 
    'no_bike_reason': ['No - I bicycle a lot', 'I live too far to bicycle', 'I do not like to bicycle', "I don't have time to bicycle", 'Fear of dangerous driving', 'Lack of adequate bike lanes', 'My health condition', 'My own personal preference', 'Visually unappealing'],
    'drive_reason': ['No - I do not drive, or prefer not to drive', "I don't have time to walk or bicycle", 'Safety concerns with walking', 'Safety concerns with bicycling', 'My own health condition', 'My own personal preference']
}

# remember that there was also an 'other' field where users could enter custom answers... we'll deal with that later.

## Fix the mistake we made splitting these columns
In the `cleanup.ipynb` notebook, we split up columns with multiple answer options into multiple columns, such as `child_drive_reason_0`, `child_drive_reason1`, etc.  Merge them back together here.

In [4]:
# loop through each question title
for q in multiple_answer_options.keys():
    column_list = []
    for i in range(10):
        column_name = q + '_' + str(i) #auto-generate numbered column name
        if column_name in df.columns:
            column_list.append(column_name)
    if len(column_list) > 0:
        # create a new column with all answers comma-separated
        df[q] = df[column_list].apply(lambda x: ', '.join(x.fillna('').map(str)), axis=1)
        df[q] = df[q].str.strip(', ')
        
        # delete the old numbered columns
        df = df.drop(column_list, axis=1)        
df.columns

Index(['date', 'neighborhood', 'demographic', 'child_bus_freq',
       'child_bike_freq', 'child_driven_freq', 'child_drive_freq',
       'child_walk_freq', 'walk_freq', 'bike_freq', 'bikes_on_sidewalk',
       'self_jog_frequency', 'commutes', 'child_self_school',
       'child_self_bus_freq', 'child_self_bike_freq', 'child_self_driven_freq',
       'child_self_drive_freq', 'child_self_walk_freq', 'child_self_commutes',
       'child_self_has_children', 'commuter_distance', 'commuter_type',
       'commuter_walk_to_station_freq', 'commuter_bike_to_station_freq',
       'commuter_drive_to_station_freq', 'commuter_carpool_to_station_freq',
       'commuter_driven_to_station_freq', 'commuter_bus_to_station_freq',
       'feelings', 'problem_areas', 'drivers_are_safe', 'bicyclists_are_safe',
       'suggested_improvements', 'additional_comments', 'owns_business',
       'business_type', 'business_space', 'business_pedestrian_synergy',
       'business_bicyclists_synergy', 'business_bike_r

## Add in the tagging data
Tagging is done by the data analyst, and is assumed to be correct.  From here on, for simplicity, it will from here be stored and analyzed the same as if tags had been a question with multiple answer options with the most popular tags being treated as the answer options.

In [5]:
#add the most popular complaint tags to the dictionary of questions and their answer options
multiple_answer_options['problem_tags'] = [ 'speeding', 'missing sidewalks', 'cpa', 'sidewalk condition', 'aggressive driving', 'driver awareness', 's riverside', '129', 'maple', 'cyclists breaking rules', 'cleveland', 'road surface', 'municipal place', 'missing crosswalks', 'bushes on sidewalk', 'grand', 'narrow roads', 'mt airy', 'shoprite', 'benedict', 'five corners', 'van wyck', 'dummy light', 'crossing guard', 'gottwald circle', 'n riverside', 'poor lighting', 'croton commons', 'truesdale', 'hmb trail', 'cet', 'on-street parking', 'old post n', 'olcott', 'old post s', 'morningside' ]

#add the most popular suggestion tags to the dictionary of questions and their answer options
multiple_answer_options['suggestion_tags'] = [ 'bike lanes', 'more sidewalks', 'enforce speed', 'maintain sidewalks', 'reduce speed', 'educate cyclists', 'more crosswalks', 'speed bumps', 'no bike lanes', 'maintain crosswalks', 'enforce crosswalks', 'sidewalk on mt airy s', 'reduce on-street parking', 'more traffic lights', 'better lighting', 'more stop signs', 'dedicated pedestrian walk signals', 'more signs', 'improve route 9 access', 'educate pedestrians', 'outlaw cycling', 'enforce trimming of bushes', 'speed cameras', 'parking on one side only', 'bike racks', 'traffic calming', 'speed indicator signs' ]


## Limit to just the multiple select columns
For debugging

In [6]:
column_list = list(multiple_answer_options.keys())
df = df[column_list]
df = df.fillna('')
df.sample(n=10) #random sample of rows

,schools,child_drive_reason,child_no_walk_reason,child_no_bike_reason,no_walk_reason,no_bike_reason,drive_reason,problem_tags,suggestion_tags
537,"PVC, CHHS",Only in times of bad weather as Mt. Airy is no...,No - they walk a lot,Our own personal preference,No - I walk a lot,I do not like to bicycle,Safety concerns with walking,"missing sidewalks, mt airy","more signs, sidewalk on mt airy s, more sidewalks"
441,"CET, PVC","No - we do not drive, or prefer not to drive",No - they walk a lot,"Fear of dangerous driving, Lack of adequate bi...",No - I walk a lot,"Fear of dangerous driving, Lack of adequate bi...","No - I do not drive, or prefer not to drive","aggressive driving,driver awareness,shoprite,m...",more sidewalks
257,Currently pre-school but soon CET,"Her pre-school is out of town, but our intenti...","Again, we intend for her to walk, though I thi...",She's just not biking yet.,No - I walk a lot,Can't bike with kiddo,My own personal preference,"municipal place,van wyck,cet,gerstein,clevelan...",
485,CET,"No - we do not drive, or prefer not to drive","We live too far to walk, Lack of adequate side...","My child is too young to bicycle, Fear of dang...",I don't have time to walk,"Fear of dangerous driving, Kids are young",Safety concerns with bicycling,"mt airy,speeding,missing sidewalks",
94,,,,,My health condition,My health condition,I don't have time to walk or bicycle,,no bike lanes
160,CET,"No - we do not drive, or prefer not to drive","Fear of dangerous driving, Lack of adequate si...","Fear of dangerous driving, Lack of adequate bi...",No - I walk a lot,"Fear of dangerous driving, Lack of adequate bi...",Safety concerns with bicycling,"benedict,cleveland,benedict,truesdale,cet,miss...","parking on one side only,reduce on-street park..."
45,,,,,My own personal preference,I don't bicycle,My own personal preference,sidewalk condition,
161,,,,,I don't have time to walk,I love to bike - it's all about time - same wi...,I don't have time to walk or bicycle,"cleveland,maple,old post s, old post n,sidewal...","more sidewalks, bike lanes,maintain sidewalks,..."
188,,,,,No - I walk a lot,Fear of dangerous driving,Safety concerns with bicycling,,
491,Happy Hearts Day Care,Safety concerns with walking,"Fear of dangerous driving, Lack of adequate si...","My child is too young to bicycle, Fear of dang...","Fear of dangerous driving, Lack of adequate si...","Fear of dangerous driving, Lack of adequate bi...","Safety concerns with walking, Safety concerns ...","grand,129,missing sidewalks,aggressive driving","more sidewalks,bike lanes"


## Remove commas from any given answer option so we can later split by comma cleanly

In [7]:
# loop through all questions that allow more than one answer
for question, answer_list in multiple_answer_options.items():
    # loop through every answer in the answer list for this question
    for answer in answer_list:
        # remove the commas, from each answer, if any
        if ',' in answer:
            # update the answers with a cleaned version
            df[question] = df[question].str.replace(answer, answer.replace(',', '') )
            
    # remove commas from the list of answers in the question/answer dictionary
    multiple_answer_options[question] = [answer.replace(',' , '') for answer in answer_list]

# show the cleaned up answer options
#multiple_answer_options

## Tally answers
Indicate which users selected which answer options, and put any freeform text answers that users entered by clicking the 'other' option in Google Forms into '_other' columns.

In [8]:
def categorize_answers(response, question, answer_list):    
    """
    Takes a given set of responses to a multi-select question, and puts them into a well-formated Series of dummy values.
    
    @param response A list of the user's responses to this question
    @param question The title of the question (these are codes)
    @param answer_list A list of the answer options that were presented to the user in the Google Form
    """
    
    # set up some lists that we will use to collect responses as dummy values
    indices = [] # will contain the text of the answer option that the user selected
    values = [] # will contain dummy values (1s) for each answer option the user selected, NaNs otherwise.
    
    # skip any nan or blank values
    if str(response) != 'nan' and type(answer_list) != float and type(question) != float and type(response) != float:
        #display(question + " - " + str(response))
        for answer_option in response:
            answer_option = answer_option.strip() # remove any leading/trailing whitespace
            if answer_option in answer_list and question + '_' + answer_option not in indices:
                indices.append(question + '_' + answer_option)
                values.append(1)
            elif answer_option.strip() != '' and question + '_' + 'other' not in indices:
                indices.append(question + '_' + 'other')
                values.append(answer_option)
    
        # put it all into a pandas Series
        response = pd.Series(values,index=indices).fillna(0).astype(str)
        
    return response

# slice up answers by comma and give each its own column
df2 = pd.DataFrame() # build a new dataframe from scratch

# loop through each question/answer item
for question, answer_list in multiple_answer_options.items():
    # get a new dataframe with the questions as indexes and the valuess as dummy values indicating user selections
    d = df[question].str.split(',\s*').apply(categorize_answers, question=question, answer_list=answer_list)
    df2 = df2.append(d) # append this as a row to the new dataframe

# show the new dataframe
df2.sample(n=10) #random sample of rows

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,child_drive_reason_Lack of available busing where we live,child_drive_reason_My child's health condition,child_drive_reason_No - we do not drive or prefer not to drive,child_drive_reason_Our own personal preference,child_drive_reason_Safety concerns with bicycling,child_drive_reason_Safety concerns with buses,child_drive_reason_Safety concerns with walking,child_drive_reason_The bus schedule does not match our schedule,child_drive_reason_other,child_no_bike_reason_Fear of dangerous driving,...,suggestion_tags_other,suggestion_tags_outlaw cycling,suggestion_tags_parking on one side only,suggestion_tags_reduce on-street parking,suggestion_tags_reduce speed,suggestion_tags_sidewalk on mt airy s,suggestion_tags_speed bumps,suggestion_tags_speed cameras,suggestion_tags_speed indicator signs,suggestion_tags_traffic calming
367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Convert values to ints or booleans, where possible
Most columns contain NaNs or 1s.  NaNs are converted to 0s then to booleans, if desired.  Integer dummy values are perhaps more useful so we can take counts of responses by adding the ints up.  The 'other' columns contain text and are left alone.

In [9]:
# loop through each column
for col in df2.columns:
    try:
        # try to convert to int
        df2[col] = df2[col].fillna(0).astype(int)#.astype(bool)
    except:
        # if this column contains text, skip...
        continue

## Group rows by 'id' so there is one row for each respondent

In [10]:
# add up all values in each column to get a full set of values or each id
# this loses the 'other' columns, since they are not numeric
df3 = df2.groupby(df2.index).sum()

# show the new grouped data... note that it is missing the 'other' fields... more on that later
df3.sample(n=10) #random sample of rows

,child_drive_reason_Lack of available busing where we live,child_drive_reason_My child's health condition,child_drive_reason_No - we do not drive or prefer not to drive,child_drive_reason_Our own personal preference,child_drive_reason_Safety concerns with bicycling,child_drive_reason_Safety concerns with buses,child_drive_reason_Safety concerns with walking,child_drive_reason_The bus schedule does not match our schedule,child_no_bike_reason_Fear of dangerous driving,child_no_bike_reason_Hills,...,suggestion_tags_no bike lanes,suggestion_tags_outlaw cycling,suggestion_tags_parking on one side only,suggestion_tags_reduce on-street parking,suggestion_tags_reduce speed,suggestion_tags_sidewalk on mt airy s,suggestion_tags_speed bumps,suggestion_tags_speed cameras,suggestion_tags_speed indicator signs,suggestion_tags_traffic calming
475,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
312,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
124,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
522,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
344,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
410,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,1,1,0,0
551,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
483,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## Combine the selected answer options with the 'other' responses users gave

In [11]:
# loop through all questions
for question, answer_options in multiple_answer_options.items():
    # get a dataframe of just the 'other' answers to this question
    other_df = df2[pd.notnull(df2[question + '_other'])][question + '_other'].to_frame()
    # concatenate those responses to the original dataframe
    df3 = df3.join(other_df)
    
# show the data
df3.sample(n=10) #random sample of rows

,child_drive_reason_Lack of available busing where we live,child_drive_reason_My child's health condition,child_drive_reason_No - we do not drive or prefer not to drive,child_drive_reason_Our own personal preference,child_drive_reason_Safety concerns with bicycling,child_drive_reason_Safety concerns with buses,child_drive_reason_Safety concerns with walking,child_drive_reason_The bus schedule does not match our schedule,child_no_bike_reason_Fear of dangerous driving,child_no_bike_reason_Hills,...,suggestion_tags_traffic calming,schools_other,child_drive_reason_other,child_no_walk_reason_other,child_no_bike_reason_other,no_walk_reason_other,no_bike_reason_other,drive_reason_other,problem_tags_other,suggestion_tags_other
83,0,0,0,0,1,1,1,1,1,0,...,0,NaN,NaN,NaN,NaN,NaN,Hilly terrain,NaN,dobbs park,NaN
271,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,Don’t know how,Distance from my airy,NaN,NaN
308,0,0,1,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raise speed limits
215,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,0,0,0,0,0,0,1,0,0,0,...,0,Circle school,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lower speed limit
475,0,0,1,0,0,0,0,0,0,0,...,0,NaN,NaN,When my child moves from CET to PVC,NaN,NaN,NaN,Trasnport of STUFF would #1 reason. i.e. groce...,NaN,NaN
66,0,0,0,1,0,0,0,0,0,0,...,0,St. Augustine,NaN,NaN,NaN,NaN,NaN,NaN,speed,stop sign on oprn at stevenson
291,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130,0,0,0,1,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,Timing,Timing I am in a rush and I have a nice bike t...,Timing and usually in a rush,taxes,lower taxes
299,0,0,0,0,0,0,0,0,0,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Order the columns alphabetically
So it looks a bit more orderly

In [12]:
column_names = df3.columns.sort_values()
df3 = df3[column_names]
df3.sample(n=10) #random sample of rows

,child_drive_reason_Lack of available busing where we live,child_drive_reason_My child's health condition,child_drive_reason_No - we do not drive or prefer not to drive,child_drive_reason_Our own personal preference,child_drive_reason_Safety concerns with bicycling,child_drive_reason_Safety concerns with buses,child_drive_reason_Safety concerns with walking,child_drive_reason_The bus schedule does not match our schedule,child_drive_reason_other,child_no_bike_reason_Fear of dangerous driving,...,suggestion_tags_other,suggestion_tags_outlaw cycling,suggestion_tags_parking on one side only,suggestion_tags_reduce on-street parking,suggestion_tags_reduce speed,suggestion_tags_sidewalk on mt airy s,suggestion_tags_speed bumps,suggestion_tags_speed cameras,suggestion_tags_speed indicator signs,suggestion_tags_traffic calming
474,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,1,0,0
268,0,0,0,0,0,0,0,0,NaN,0,...,retrain crossing guards,0,0,0,0,0,0,0,0,0
321,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
407,0,0,0,1,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
468,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
234,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
334,0,0,0,0,0,0,0,0,NaN,0,...,NaN,0,0,0,0,0,0,0,0,0
51,0,0,0,0,0,0,0,0,NaN,0,...,allow scooters in bike lanes,0,0,0,0,0,0,0,0,0
27,0,0,1,0,0,0,0,0,NaN,1,...,NaN,0,0,0,0,0,0,0,0,0
102,0,0,0,0,0,0,0,0,NaN,0,...,NaN,1,0,0,0,0,0,0,0,0


## Merge with remaining data

In [13]:
# load up the original data from scratch again
df = pd.read_csv('responses_scrubbed_tagged.csv', index_col=None, parse_dates=['date'])

# merge the new columns with the old dataframe based on the indexes of each row
df_complete = pd.merge(df, df3, right_index=True, left_index=True)

df_complete.sample(n=10) #random sample of rows

,date,neighborhood,demographic,child_bus_freq,child_bike_freq,child_driven_freq,child_drive_freq,child_walk_freq,walk_freq,bike_freq,...,suggestion_tags_other,suggestion_tags_outlaw cycling,suggestion_tags_parking on one side only,suggestion_tags_reduce on-street parking,suggestion_tags_reduce speed,suggestion_tags_sidewalk on mt airy s,suggestion_tags_speed bumps,suggestion_tags_speed cameras,suggestion_tags_speed indicator signs,suggestion_tags_traffic calming
471,2019-07-10 22:07:31,Upper Village,Adult without school kid,NaN,NaN,NaN,NaN,NaN,"Every day, or almost every day",Once in a while,...,NaN,0,0,0,1,0,0,0,0,0
478,2019-07-11 16:48:19,Old Post Road North,Adult with school kid,"Every day, or almost every day",Rarely or never - for other reasons,Once in a while,Rarely or never - my children are too young,Rarely or never - for other reasons,Once every few days,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0
131,2019-07-03 17:07:37,Mount Airy,Senior,NaN,NaN,NaN,NaN,NaN,Once every few days,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0
124,2019-07-03 16:55:00,Harmon,Adult with school kid,"Every day, or almost every day",Rarely or never - for other reasons,Rarely or never,Rarely or never - for other reasons,Rarely or never - for other reasons,Rarely or never,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0
421,2019-07-09 12:39:05,Upper Village,Adult with school kid,"Every day, or almost every day",Rarely or never - my children are too young,Once in a while,Rarely or never - my children are too young,Rarely or never - my children are too young,"Every day, or almost every day","Every day, or almost every day",...,NaN,0,0,0,0,0,0,0,0,0
219,2019-07-03 21:53:12,Old Post Road North,Adult with school kid,"Every day, or almost every day",Rarely or never - my children are too young,Once in a while,Rarely or never - my children are too young,Rarely or never - my children are too young,Once every few days,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0
114,2019-07-03 16:37:06,Upper Village,Adult with school kid,Rarely or never,Rarely or never - for other reasons,Once every few days,Rarely or never - my children are too young,"Every day, or almost every day","Every day, or almost every day",Once in a while,...,NaN,0,0,0,0,0,0,0,0,0
176,2019-07-03 19:02:48,Sunset Park,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Once in a while,Once in a while,...,NaN,0,0,0,0,0,0,0,0,0
20,2019-07-03 15:37:33,Old Post Road North,Adult without school kid,NaN,NaN,NaN,NaN,NaN,Once in a while,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0
227,2019-07-03 22:58:41,Sunset Park,Adult with school kid,"Every day, or almost every day",Rarely or never - my children are too young,Rarely or never,Rarely or never - my children are too young,Rarely or never - my children are too young,Once in a while,Rarely or never,...,NaN,0,0,0,0,0,0,0,0,0


## Group together some related sets of tags from the free text responses
There were several tags given for complaints about speeding, sidewalks, crosswalks, etc.... Make aggregate columns to make these things easier to search for.

In [14]:
def aggregate_tags(row, related_tags):
    """
    This method looks for a dummy value in a set of related fields.
    Returns 1 if found, 0 others.
    """
    # loop through each tag in the group
    for tag in related_tags:
        # if we find a one, then we're done
        if row[tag] == 1:
            return 1
    # otherwise, we did not find what we were looking for
    return 0

# make some new aggregate columns around topics that are otherwise represented across several fields

# a bunch of tags all related to concerns with drivers' behavior and speed
related_tags = ['problem_tags_driver awareness', 'problem_tags_aggressive driving', 'problem_tags_speeding', 'suggestion_tags_reduce speed', 'suggestion_tags_enforce speed', 'suggestion_tags_speed bumps', 'suggestion_tags_speed cameras', 'suggestion_tags_speed indicator signs', 'suggestion_tags_traffic calming']
df_complete['concerns_with_drivers'] = df_complete.apply (lambda row: aggregate_tags(row, related_tags), axis=1)

# a bunch of tags all related to concerns with sidewalks
related_tags = ['problem_tags_missing sidewalks', 'problem_tags_sidewalk condition', 'suggestion_tags_more sidewalks', 'suggestion_tags_maintain sidewalks']
df_complete['concerns_with_sidewalks'] = df_complete.apply (lambda row: aggregate_tags(row, related_tags), axis=1)

# a bunch of tags all related to concerns with crosswalks
related_tags = ['problem_tags_missing crosswalks', 'suggestion_tags_more crosswalks', 'suggestion_tags_maintain crosswalks', 'suggestion_tags_enforce crosswalks']
df_complete['concerns_with_crosswalks'] = df_complete.apply (lambda row: aggregate_tags(row, related_tags), axis=1)


## Save to new CSV file

In [15]:
df_complete.to_csv('responses_complete.csv', index=False)

## Done!